In [1]:
import json

from google.colab import files

license_keys = files.upload()

with open(list(license_keys.keys())[0]) as f:
    license_keys = json.load(f)

Saving spark_ocr.json to spark_ocr.json


In [2]:
%%capture
for k,v in license_keys.items(): 
    %set_env $k=$v

In [3]:
%%bash
if python -c 'import google.colab' &> /dev/null; then
    echo "Run on Google Colab!"
    echo "Install Open JDK"
    apt-get install -y openjdk-8-jdk-headless -qq > /dev/null
    java -version
fi

Run on Google Colab!
Install Open JDK


openjdk version "11.0.11" 2021-04-20
OpenJDK Runtime Environment (build 11.0.11+9-Ubuntu-0ubuntu2.18.04)
OpenJDK 64-Bit Server VM (build 11.0.11+9-Ubuntu-0ubuntu2.18.04, mixed mode, sharing)


In [6]:
import sys
import os

version = os.environ["JSL_OCR_SECRET"].split("-")[0]
secret = os.environ["JSL_OCR_SECRET"]

if 'google.colab' in sys.modules:
  os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
  os.environ["PATH"] = os.environ["JAVA_HOME"] + "/bin:" + os.environ["PATH"]

In [7]:
%pip install spark-ocr==$version+spark30 --extra-index-url=https://pypi.johnsnowlabs.com/$secret --upgrade

Looking in indexes: https://pypi.org/simple, https://pypi.johnsnowlabs.com/3.6.0-609517f381c67278bd23d63fa911d22b3a3b1b83
     |████████████████████████████████| 16.2 MB 1.4 MB/s 
     |████████████████████████████████| 198 kB 6.6 MB/s 
     |████████████████████████████████| 204.8 MB 51 kB/s 
     |████████████████████████████████| 29.2 MB 45 kB/s 
     |████████████████████████████████| 2.2 MB 29.9 MB/s 
  Created wheel for pyspark: filename=pyspark-3.0.2-py2.py3-none-any.whl size=205186690 sha256=7473d4d0a8ba7ee4132d52e0232c68ebffce8d4c45a46324bf7c3b1b6e8d010d
  Stored in directory: /root/.cache/pip/wheels/9a/39/f6/970565f38054a830e9a8593f388b36e14d75dba6c6fdafc1ec
Successfully built pyspark
  Attempting uninstall: pillow
    Found existing installation: Pillow 7.1.2
    Uninstalling Pillow-7.1.2:
      Successfully uninstalled Pillow-7.1.2
  Attempting uninstall: scikit-image
    Found existing installation: scikit-image 0.16.2
    Uninstalling scikit-image-0.16.2:
      Successful

In [9]:
from sparkocr import start
import json
import os
from pyspark.ml import Pipeline
from pyspark.sql import SparkSession

import pandas as pd

spark = start(secret=secret)
spark

Spark version: 3.0.2
Spark OCR version: 3.6.0



In [12]:
from sparkocr.transformers import *

pdfPath = "a.pdf"

# Read PDF file as binary file
df = spark.read.format("binaryFile").load(pdfPath)

transformer = PdfToText() \
  .setInputCol("content") \
  .setOutputCol("text") \
  .setPageNumCol("pagenum") \
  .setSplitPage(True)

data_light = transformer.transform(df)

data_light.select("pagenum", "text").show(2000,False)

+-------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [11]:
from sparkocr.transformers import *

pdfPath = "a.pdf"

# Read PDF file as binary file
df = spark.read.format("binaryFile").load(pdfPath)

# Transform PDF file to the image
pdfToImage = PdfToImage() \
  .setInputCol("content") \
  .setOutputCol("image")

# OCR
ocr = ImageToText() \
  .setInputCol("image") \
  .setOutputCol("text")

# Define pipeline
pipeline = Pipeline(stages=[
  pdfToImage,
  ocr
])

data = pipeline.fit(df).transform(df)

data.select("text").show(2000,False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [13]:
data_light.select("text").write.save("data_light.parquet")

In [14]:
data.select("text").write.save("data.parquet")